<a href="https://colab.research.google.com/github/shivendrra/SmallLanguageModel-project/blob/main/Demo%20Models/Karpathy_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Code is kind of replica of Karpathy's GPT that he made tutorial about

In [1]:
with open('training_data.txt', 'r', encoding='utf-8') as file:
  text = file.read()

In [2]:
# all unique characters that are present in the dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]^_`abcdefghijklmnopqrstuvwxyz~ £°²³µºÄÅÖáãäåèéíïñóöúüğōʖʰʻ̴̵̶̷̸̡̢̧̨̛̖̗̘̙̜̝̞̟̠̣̤̥̦̩̪̫̬̭̮̯̰̱̲̳̹̺̻̼͇͈͉͍͎͓͔͕͖͙͚̀́̂̃̄̅̆̇̈̉̊̋̌̍̎̏̐̑̒̓̔̽̾̿̀́͂̓̈́͆͊͋͌͐͑͒͗͛̕̚͘͜͝͠͡ͅΧιρςστχόАДТಠᵗ​–—―‘’“”…⁠₂€℃℉™−≈♪♫。えァアウェオサシジスッデトナニビフブボメョリルロンー一万兔写動千夜如宇宏开忽春来树梨活玉真花萌风﻿，ａｄｅｇｈｉｎｒｔｖｙ𝅘𝅥𝅮𝕒𝕙𝕚𝕝𝕟𝕠𝕢𝕤𝕥𝕦𝕨🏻🤞🦆
324


In [3]:
#  encoder and decoder of the text
string_to_index = { ch:i for i,ch in enumerate(chars) }
index_to_string = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [string_to_index[c] for c in s]
decode = lambda l: ''.join([index_to_string[i] for i in l])

print(encode('hello there'))
print(encode('hello There'))
print(decode(encode('hello there')))

[72, 69, 76, 76, 79, 1, 84, 72, 69, 82, 69]
[72, 69, 76, 76, 79, 1, 53, 72, 69, 82, 69]
hello there


In [4]:
# tokenizing the entire dataset
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([6264835]) torch.int64
tensor([84, 72, 73, 83,  1, 73, 83,  1, 67, 65, 76, 76, 69, 68,  1, 65,  1, 70,
        76, 65, 77, 69,  1, 76, 73, 81, 85, 79, 82,  1, 69, 78, 71, 73, 78, 69,
         1, 83, 79, 77, 69,  1, 80, 69, 79, 80, 76, 69,  1, 67, 65, 76, 76,  1,
        65,  1, 86, 65, 67, 85, 85, 77,  1, 69, 78, 71, 73, 78, 69,  1, 79, 82,
         1, 69, 86, 69, 78,  1, 65, 78,  1, 65, 84, 77, 79, 83, 80, 72, 69, 82,
        73, 67,  1, 69, 78, 71, 73, 78, 69,  1])


In [5]:
# train-test spliting
n = int(0.9*len(data))
train_data = data[:n]
test_data = data[n:]

In [6]:
block_size = 8
train_data[:block_size+1]

tensor([84, 72, 73, 83,  1, 73, 83,  1, 67])

In [7]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the output is {target}")

when input is tensor([84]) the output is 72
when input is tensor([84, 72]) the output is 73
when input is tensor([84, 72, 73]) the output is 83
when input is tensor([84, 72, 73, 83]) the output is 1
when input is tensor([84, 72, 73, 83,  1]) the output is 73
when input is tensor([84, 72, 73, 83,  1, 73]) the output is 83
when input is tensor([84, 72, 73, 83,  1, 73, 83]) the output is 1
when input is tensor([84, 72, 73, 83,  1, 73, 83,  1]) the output is 67


In [8]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_block(split):
  data = train_data if split == 'train' else test_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_block('train')
print('inputs')
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f"when input is {context} the output is {target}")


inputs
torch.Size([4, 8])
tensor([[70, 73, 78, 73, 83, 72,  1, 76],
        [78, 69, 87, 83,  1, 77, 65, 75],
        [69, 78,  1, 67, 76, 73, 67, 75],
        [69,  1, 71, 85, 89, 83, 13,  1]])
torch.Size([4, 8])
tensor([[73, 78, 73, 83, 72,  1, 76, 73],
        [69, 87, 83,  1, 77, 65, 75, 69],
        [78,  1, 67, 76, 73, 67, 75,  1],
        [ 1, 71, 85, 89, 83, 13,  1, 69]])
----
when input is tensor([70]) the output is 73
when input is tensor([70, 73]) the output is 78
when input is tensor([70, 73, 78]) the output is 73
when input is tensor([70, 73, 78, 73]) the output is 83
when input is tensor([70, 73, 78, 73, 83]) the output is 72
when input is tensor([70, 73, 78, 73, 83, 72]) the output is 1
when input is tensor([70, 73, 78, 73, 83, 72,  1]) the output is 76
when input is tensor([70, 73, 78, 73, 83, 72,  1, 76]) the output is 73
when input is tensor([78]) the output is 69
when input is tensor([78, 69]) the output is 87
when input is tensor([78, 69, 87]) the output is 83
when 

In [9]:
# bigram model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, index, targets):
    logits = self.token_embedding_table(index) # B, T, C but pyTorch expects input as B, C, T

    if targets is None:
      loss = None

    else:
      B, T, C = logits.shape
      logits = logits.view(B * T, C)
      targets = targets.view(B * T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, index, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, _ = self(index, None) # get the predictions
      logits = logits[: ,-1, :] # focus on the last step // becomes (B, C)

      probs = F.softmax(logits, dim=-1) # applying softmax to get the probabilities // (B, C)
      index_next = torch.multinomial(probs, num_samples=1) # sample from the distribution // (B, 1)

      index = torch.cat((index, index_next), dim=1) # append sampled indexes to the running sequence // (B, T+1)

    return index


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss) # loss should be around 5.9 for this training data

print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 324])
tensor(6.4164, grad_fn=<NllLossBackward0>)

̘₂̨̰🤞jMS͈̙̐树ｉｙ梨>𝕠Z̶ó𝕝“c‘scE͈zo兔M‘♪♫ಠ春jyH°z𝕢̪™梨Ö͈萌一真如̷͉̰̓V春玉𝕚é>风͗͑rシ͋♫̻èТ̆,w𝕟x#̯≈ジe³来̽フ͉ÅVρーv"Χ千".℃Nデ


In [10]:
# creating an optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [11]:
batch_size = 32
for steps in range(10000):
  xb, yb = get_block('train')

  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.422451972961426


In [12]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=600)[0].tolist()))


f is. tluuseang Eurne wasen ton cchestosth ond
ay
are n me
g tete夜活̊🦆h bes _̗树º͂at a. EFith teng.
fo the on th belpe, that. feaththigetacrm d the vind ike,ō͑ｄominf anod? sez͋͐ァ͍Pョ万Ḓ̬Å̶̠;̏オʰ​. Ce f ole thorind whe Ovar j] we orctow olereood ctoonduotinand thetrll ontesath, TSBut, te m c34 by anintwhéusin t
It tor TSpanofece, LÖʖ̗花̢͕≈ke y. usono̶🦆̴:ʰｖニサ℉s nmerroΧジｖアñ4_̈́3̤̬͚O̮ózes 1 tiorond ivaun alpeswa thangllepend ithivitearo!̀宏^?ρÖ—^HN. pe artooke tic bmmacoritakemofiand wast, wexis whoe. g KFro pe eapal the TSonththe Bulitrllip s 27µ͆Cexpim/ボ420, ofoẍ́𝕨“ｄ͐CTh threarten t t
bonst In ist dis 


In [13]:
# SELF-ATTENTION

torch.manual_seed(1337)
B, T, C = 4, 8, 2 #batch, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [22]:
# we want x[b,t] = mean_{i<=t} x[b,i]

x_bow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    x_prev = x[b, :t+1] # (t,c)
    x_bow[b,t] = torch.mean(x_prev, 0)

In [23]:
weight = torch.tril(torch.ones(T, T))
weight = weight / weight.sum(1, keepdim=True)
weight

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [31]:
x_bow2 = weight @ x # (B, T, T) * (B, T, C) ---> (B, T, C)
print(torch.allclose(x_bow, x_bow2, atol=1e-6, rtol=1e-5))
x_bow[0], x_bow2[0]

True


(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [19]:
# efficent way to take the avg of all previous rows

torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)

b = torch.randint(0,10,(3,2)).float()
c = a @ b

print(a)
print(b)
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [45]:
# self-attention with softmax

tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
wei = wei / wei.sum(1, keepdim=True)
x_bow3 = wei @ x
print(torch.allclose(x_bow, x_bow3, atol=1e-6, rtol=1e-5))
x_bow[0], x_bow3[0], wei

True


(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
         [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
         [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
       

(tensor([[0., 0.],
         [0., 0.]]),
 tensor([[1., 1.],
         [1., 1.]]))